In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB,GaussianNB,BernoulliNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from math import sqrt
from sklearn.svm import SVC
import time
import tracemalloc
import xgboost as xgb

In [2]:
df=pd.read_csv('restaurant.csv')

In [3]:
cv = CountVectorizer()
cv.fit(df['review'])
X = cv.transform(df['review'])

In [4]:
LogReg_clf = LogisticRegression()
SVC_clf = SVC(probability=True, kernel='rbf')
NV=MultinomialNB()
NK=BernoulliNB()
DTree_clf = DecisionTreeClassifier()
svm=LinearSVC()
rf = RandomForestClassifier(random_state=1200, criterion='entropy', n_estimators=200)
#ada = AdaBoostClassifier(n_estimators=200, learning_rate=2.0, algorithm='SAMME')
ada = AdaBoostClassifier()
ETree = ExtraTreesClassifier(random_state=42)

In [5]:
msg_train, msg_test, label_train, label_test =train_test_split(X, df['label'], test_size=0.2,random_state=42)

In [10]:
# get the start time
st = time.time()
tracemalloc.start()


rf.fit(msg_train,label_train)
preds= rf.predict(msg_test)
acc = accuracy_score(label_test, preds)
print("Accuracy For DTree Model is: " + str(acc))
rms= sqrt(mean_squared_error(label_test, preds))
print("RMSE Error is: " + str(rms))


# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

current, peak = tracemalloc.get_traced_memory()
# Stop tracemalloc
tracemalloc.stop()
#print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
print("Current memory usage is", current / (1024 * 1024), "MB; Peak was", peak / (1024 * 1024), "MB")

Accuracy For DTree Model is: 0.9509615384615384
RMSE Error is: 0.2214462949305351
Execution time: 19.85596990585327 seconds
Current memory usage is 11.829256057739258 MB; Peak was 13.701107025146484 MB


In [11]:
#Voting Ensemble Method
# get the start time
st = time.time()
tracemalloc.start()



#voting_clf = VotingClassifier(estimators=[('LR',model),('NV', ada)], voting='hard')
voting_clf = VotingClassifier(estimators=[('svm', SVC_clf), ('AdaBoost', ada), ('RF',rf )], voting='soft')
voting_clf.fit(msg_train, label_train)
preds = voting_clf.predict(msg_test)

acc = accuracy_score(label_test, preds)
rms= sqrt(mean_squared_error(label_test, preds))


print("Accuracy For Voting Ensemble SVM+AadBoost+RF is: " + str(acc))
print("RMSE Error is: " + str(rms))


# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

current, peak = tracemalloc.get_traced_memory()
# Stop tracemalloc
tracemalloc.stop()
print("Current memory usage is", current / (1024 * 1024), "MB; Peak was", peak / (1024 * 1024), "MB")

Accuracy For Voting Ensemble SVM+AadBoost+RF is: 0.95625
RMSE Error is: 0.2091650066335189
Execution time: 44.53401780128479 seconds
Current memory usage is 0.9448156356811523 MB; Peak was 6.826879501342773 MB


In [12]:
# Boosting Ensemble (Adaboost classifier)

# get the start time
st = time.time()
tracemalloc.start()

clf = AdaBoostClassifier()
clf.fit(msg_train, label_train)

# make predictions on the testing set
y_pred = clf.predict(msg_test)

# evaluate the model's performance using accuracy and RMSE
accuracy = accuracy_score(label_test, y_pred)
rmse = mean_squared_error(label_test, y_pred)

print("Accuracy for AdaBoost:", accuracy)
print("RMSE:", rmse)

# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

current, peak = tracemalloc.get_traced_memory()
# Stop tracemalloc
tracemalloc.stop()
print("Current memory usage is", current / (1024 * 1024), "MB; Peak was", peak / (1024 * 1024), "MB")

Accuracy for AdaBoost: 0.9355769230769231
RMSE: 0.06442307692307692
Execution time: 3.38315749168396 seconds
Current memory usage is 0.054737091064453125 MB; Peak was 1.9935922622680664 MB


In [13]:
# Train XGBoost model
# get the start time
st = time.time()
tracemalloc.start()


model = xgb.XGBClassifier(n_estimators=70, learning_rate=0.9)
model.fit(msg_train, label_train)

# Predict on test set and calculate accuracy
y_pred = model.predict(msg_test)
acc = accuracy_score(label_test, y_pred)
print('Accuracy for XGBoost:', acc)

# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

current, peak = tracemalloc.get_traced_memory()
# Stop tracemalloc
tracemalloc.stop()
#print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
print("Current memory usage is", current / (1024 * 1024), "MB; Peak was", peak / (1024 * 1024), "MB")

Accuracy for XGBoost: 0.9514423076923076
Execution time: 0.4694514274597168 seconds
Current memory usage is 0.1057443618774414 MB; Peak was 0.17767333984375 MB
